In [2]:
import sounddevice as sd
import numpy as np
import wave
import openai
import fitz  # PyMuPDF

openai.api_key = 'API_KEY'

# 오디오 녹음 함수
def record_audio(filename='presentation.wav', duration=60, fs=44100):
    print("녹음을 시작합니다...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=2, dtype='int16')
    sd.wait()  # 녹음이 끝날 때까지 대기
    print("녹음이 완료되었습니다.")
    
    # WAV 파일로 저장
    with wave.open(filename, 'w') as wf:
        wf.setnchannels(2)
        wf.setsampwidth(2)  # 16-bit
        wf.setframerate(fs)
        wf.writeframes(recording.tobytes())
    
    return filename

# Whisper를 사용하여 오디오 파일을 텍스트로 변환
def transcribe_audio(file_path):
    with open(file_path, "rb") as audio_file:
        response = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file
        )
    return response['text']

# PDF에서 텍스트 추출
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# 텍스트 비교 및 피드백 제공
def generate_feedback(pdf_text, presentation_text):
    prompt = f"""
    아래는 논문 PDF에서 추출한 텍스트와 발표자의 음성 파일을 텍스트로 변환한 내용입니다. 두 텍스트를 비교하여, 문맥의 일관성, 주요 주제의 정확성, 그리고 발표의 명확성에 대해 피드백을 제공해 주세요. 또한, 발표자가 개선할 수 있는 부분에 대한 제언도 포함해 주세요. 모든 피드백은 한국어로 작성해 주세요.

    논문 PDF 텍스트:
    {pdf_text}

    발표 텍스트:
    {presentation_text}

    피드백:
    """
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=1000,
        temperature=0.7,
        n=1,
        stop=None
    )
    return response.choices[0].text

# 텍스트 비교 및 예상 질문 생성
def generate_comparison_questions(pdf_text, presentation_text):
    prompt = f"""
    아래는 논문 PDF에서 추출한 텍스트와 발표자의 음성 파일을 텍스트로 변환한 내용입니다. 두 텍스트를 비교하여, 발표 내용을 기반으로 예상 질문을 5개 작성해 주세요. 모든 질문은 한국어로 작성해 주세요.

    논문 PDF 텍스트:
    {pdf_text}

    발표 텍스트:
    {presentation_text}

    예상 질문:
    """
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=1000,
        temperature=0.7,
        n=1,
        stop=None
    )
    return response.choices[0].text

# 메인 인터랙션 함수
def chat_bot():
    # 1. 녹음을 시작한다.
    # duration = int(input("녹음 시간을 입력하세요 (초): "))
    check_audio = input("녹음을 시작하시겠습니까? (예/아니오): ").strip().lower()
    if check_audio == '예':
        audio_file = record_audio(duration=120)

    else:
        print("대화를 종료합니다.")
        # 끝내기
        return   

    # 2. 녹음이 끝나면 pdf 를 제출할 것이냐는 물음을 한다.
    submit_pdf = input("PDF를 제출하시겠습니까? (예/아니오): ").strip().lower()

    if submit_pdf == '예' or submit_pdf == '제출':
        pdf_path = "./research_2.pdf"
        # 3. pdf 파일을 받아왔다면 "pdf 제출 완료" 라는 말과 함께 "1. 피드백 받기  2. 예상 질문 받기" 를 선택하도록 묻는다.
        print("PDF 제출 완료")
        
        action = input("원하는 작업을 선택하세요: 1. 피드백 받기  2. 예상 질문 받기: ").strip()
        pdf_text = extract_text_from_pdf(pdf_path)
        presentation_text = transcribe_audio(audio_file)
        
        if action == '1' or action == '피드백' or action == '1. 피드백 받기' or action == '피드백 받기':
            feedback = generate_feedback(pdf_text, presentation_text)
            print("피드백: \n", feedback)
        elif action == '2' or action == '예상 질문' or action == '2. 예상 질문 받기' or action == '예상 질문 받기':
            questions = generate_comparison_questions(pdf_text, presentation_text)
            print("예상 질문: \n", questions)
        else:
            print("잘못된 입력입니다. 대화를 종료합니다.")
            return
        
        # 6. 또 다른 질문이 있는지 물어본다.
        another_question = input("또 다른 질문이 있으신가요? (예/아니오): ").strip().lower()
        if another_question == '예':
            chat_bot()  # 재귀 호출로 다시 시작
        else:
            print("대화를 종료합니다.")
    else:
        # 7. 없다하면 대화 종료
        print("대화를 종료합니다.")

# 챗봇 실행
chat_bot()


녹음을 시작합니다...
녹음이 완료되었습니다.
PDF 제출 완료


AttributeError: type object 'Audio' has no attribute 'transcriptions'